In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import threading
import time
import os

# Basis-URL
base_url = 'https://waarneming.nl/fieldwork/observations/daylist/'

# Definieer jaar
year = 2024

# Pad waar CSV-bestanden worden opgeslagen
output_dir = "waarnemingen_data"

# Maak de directory aan als deze niet bestaat
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Directory '{output_dir}' aangemaakt.")

# Lijst met species_groups
species_groups = [10, 19, 12, 11]  # Planten, Algen, Mossen, Paddenstoelen

# Functie om een specifieke pagina te scrapen
def scrape_page(date, page, species_group, data):
    url = f"{base_url}?date={date}&species_group={species_group}&search=Rotterdam&page={page}"
    print(f"Scraping pagina {page} voor {date} - Species {species_group}: {url}")

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Fout: Kon pagina {page} niet laden voor species {species_group} (statuscode {response.status_code})")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', class_='table table-bordered table-striped')

    if not table:
        print(f"Geen tabel gevonden op pagina {page} voor species {species_group}.")
        return

    rows = table.find_all('tr')
    if len(rows) <= 1:
        print(f"Geen data meer gevonden voor {date} - Species {species_group}.")
        return

    # Extract headers
    headers = [header.text.strip() for header in rows[0].find_all('th')]

    for row in rows[1:]:
        cols = row.find_all('td')
        if len(cols) == len(headers):
            # Haal de gewenste kolommen op
            soort = cols[3].text.strip()  # Soort (kolom 3)
            locatie = cols[4].text.strip()  # Locatie (kolom 4)
            waarnemer = cols[5].text.strip()  # Waarnemer (kolom 5)
            datum = date  # Gebruik de datum van de URL

            # Voeg de data toe aan de lijst
            data.append([datum, soort, locatie, waarnemer])

# Functie om data voor een specifieke dag te scrapen
def scrape_day(date, species_group, data):
    print(f"Scraping data voor {date} - Species {species_group}...")
    page = 1
    while True:
        scrape_page(date, page, species_group, data)
        if not data:
            break
        page += 1

# Functie om data voor een specifieke maand te scrapen
def scrape_month(year, month, use_threading=False):
    start_date = datetime(year, month, 1)
    end_date = (datetime(year, month + 1, 1) - timedelta(days=1)) if month < 12 else datetime(year, 12, 31)

    print(f"Scraping data voor {start_date.strftime('%Y-%m-%d')} tot {end_date.strftime('%Y-%m-%d')}...")

    for species_group in species_groups:
        data = []
        print(f"Scrapen voor species_group {species_group} in {start_date.strftime('%Y-%m')}...")

        if use_threading:
            day_threads = []
            current_date = start_date
            while current_date <= end_date:
                date_str = current_date.strftime('%Y-%m-%d')
                thread = threading.Thread(target=scrape_day, args=(date_str, species_group, data))
                day_threads.append(thread)
                thread.start()
                time.sleep(1)  # Voorkom overbelasting van de server
                current_date += timedelta(days=1)

            for thread in day_threads:
                thread.join()
        else:
            current_date = start_date
            while current_date <= end_date:
                date_str = current_date.strftime('%Y-%m-%d')
                scrape_day(date_str, species_group, data)
                current_date += timedelta(days=1)

        # Verwijder dubbele rijen
        if data:
            df = pd.DataFrame(data, columns=['Datum', 'Soort', 'Locatie', 'Waarnemer'])
            df.drop_duplicates(inplace=True)  # Verwijder dubbele rijen

            # Sla data op als CSV per species_group
            filename = os.path.join(output_dir, f"waarnemingen_{year}_{month:02d}_species_{species_group}.csv")
            df.to_csv(filename, index=False)
            print(f"Data opgeslagen als {filename}")
        else:
            print(f"Geen data voor species_group {species_group} in {start_date.strftime('%Y-%m')}.")

# Vraag de gebruiker om een keuze
choice = input("Wil je alle maanden scrapen of een specifieke maand? (Typ 'alle' of een maandnummer (1-12)): ").strip().lower()

# Lijst voor threads
threads = []

if choice == 'alle':
    for month in range(1, 13):
        thread = threading.Thread(target=scrape_month, args=(year, month, True))  # Threading ingeschakeld
        threads.append(thread)
        thread.start()
        time.sleep(1)
elif choice.isdigit() and 1 <= int(choice) <= 12:
    scrape_month(year, int(choice), use_threading=True)  # Threading ingeschakeld
else:
    print("Ongeldige invoer. Voer 'alle' in of een maandnummer (1-12).")

if choice == 'alle':
    for thread in threads:
        thread.join()

print("Scraping voltooid.")

Scraping data voor 2024-01-01 tot 2024-01-31...
Scrapen voor species_group 10 in 2024-01...
Scraping data voor 2024-01-01 - Species 10...
Scraping pagina 1 voor 2024-01-01 - Species 10: https://waarneming.nl/fieldwork/observations/daylist/?date=2024-01-01&species_group=10&search=Rotterdam&page=1
Scraping pagina 2 voor 2024-01-01 - Species 10: https://waarneming.nl/fieldwork/observations/daylist/?date=2024-01-01&species_group=10&search=Rotterdam&page=2
Scraping pagina 3 voor 2024-01-01 - Species 10: https://waarneming.nl/fieldwork/observations/daylist/?date=2024-01-01&species_group=10&search=Rotterdam&page=3
Scraping pagina 4 voor 2024-01-01 - Species 10: https://waarneming.nl/fieldwork/observations/daylist/?date=2024-01-01&species_group=10&search=Rotterdam&page=4
Scraping pagina 5 voor 2024-01-01 - Species 10: https://waarneming.nl/fieldwork/observations/daylist/?date=2024-01-01&species_group=10&search=Rotterdam&page=5
Scraping data voor 2024-01-02 - Species 10...
Scraping pagina 1 voo